# Análise de dados - Open Observatory of Network Interference (OONI) 

# Análise de Dados – Camada Gold (OONI)

## 1. Introdução

Esta etapa corresponde à **fase de Análise** do MVP, conforme definido nos requisitos do trabalho. A análise é realizada exclusivamente sobre a **Camada Gold**, construída a partir de um pipeline de Engenharia de Dados em Databricks/Spark, com dados do **Open Observatory of Network Interference (OONI)**.

A camada Gold foi modelada em **Esquema Estrela**, permitindo consultas analíticas via SQL com foco em:

* Qualidade dos dados
* Comparação regional de censura
* Impacto temporal de eventos
* Identificação de provedores (ASNs) com maior incidência de anomalias

Todas as análises aqui apresentadas estão diretamente conectadas às **perguntas de negócio** definidas na fase de Objetivo.

---

## 2. Análise de Qualidade dos Dados

Antes de responder às perguntas de negócio, foi realizada uma análise de qualidade dos dados, com foco na **completude dos principais atributos analíticos**.

### 2.1 Métrica de Completude

A tabela `gold_data_quality_summary` consolida a taxa de valores ausentes nos principais identificadores dimensionais da tabela fato (`gold_fact_measurement`).

A análise considera:

* `country_code`: país da medição
* `asn`: provedor de internet

Esses campos são críticos para as análises regionais e de provedores.

### 2.2 Resultados

Os resultados mostram:

* A **quantidade total de registros** disponíveis para análise
* A **taxa percentual de valores ausentes** por atributo

A baixa incidência de valores nulos indica que:

* O processo de coleta via API do OONI é consistente
* As transformações aplicadas na camada Silver (padronização, tipagem e coalescência) foram eficazes

### 2.3 Impacto na Análise

Os níveis de completude observados **não comprometem as análises subsequentes**, permitindo:

* Comparações regionais confiáveis
* Identificação consistente de ASNs
* Análises temporais sem vieses estruturais

---

## 3. Solução do Problema

Esta seção responde diretamente às perguntas de negócio definidas no objetivo do MVP.

---

## 3.1 Comparação de Censura Regional

**Pergunta:**

> Quais são os sites e/ou aplicativos mais consistentemente bloqueados ou com maior taxa de anomalias por país na América Latina no último ano? A taxa observada no Brasil está acima ou abaixo da média regional?

### Abordagem Analítica

Foi utilizada a tabela `gold_censorship_by_country_test`, que agrega:

* Total de medições
* Quantidade de medições anômalas
* Taxa de anomalias (`anomaly_rate`)

O recorte temporal considera os **últimos 12 meses**, garantindo uma análise atualizada.

Em seguida, a tabela `gold_brazil_vs_region` consolida:

* A média regional da taxa de anomalias por teste
* A taxa específica do Brasil

### Discussão dos Resultados

A análise permite observar que:

* Certos aplicativos (como mensageria) apresentam taxas de anomalia consistentemente maiores em alguns países da região
* O Brasil, em determinados testes, apresenta taxas **alinhadas ou abaixo da média regional**, indicando menor incidência relativa de interferência
* Em outros casos pontuais, a taxa brasileira se aproxima da média, sugerindo eventos específicos ou interferências localizadas

Essa comparação regional evidencia o valor do modelo analítico construído, permitindo contextualizar o cenário brasileiro dentro da América Latina.

---

## 3.2 Impacto Temporal de Bloqueios

**Pergunta:**

> Existe correlação entre períodos de grandes eventos cívicos/políticos e picos no volume de medições de censura?

### Abordagem Analítica

Foi criada a tabela `gold_blocking_over_time`, agregando:

* Total de medições por data e país
* Quantidade de bloqueios confirmados (`confirmed_blocks`)

Essa estrutura permite análises temporais granulares, por dia.

### Discussão dos Resultados

A análise temporal revela:

* Picos pontuais no número de bloqueios confirmados em determinados períodos
* Esses picos podem ser cruzados, em análises futuras, com calendários de eventos políticos, eleições ou protestos

Embora este MVP não incorpore dados externos de eventos cívicos, o modelo construído **viabiliza diretamente esse enriquecimento**, demonstrando a robustez da arquitetura proposta.

---

## 3.3 Análise de Provedores de Internet (Brasil)

**Pergunta:**

> Quais provedores (ASNs) no Brasil registram maior percentual de resultados anômalos?

### Abordagem Analítica

Foi criada a tabela `gold_blocking_by_asn_br`, que calcula, para o Brasil:

* Total de medições por ASN
* Quantidade de anomalias
* Taxa de anomalias

Os resultados são ordenados pela maior taxa de anomalia.

### Discussão dos Resultados

A análise evidencia que:

* Alguns ASNs específicos concentram taxas significativamente mais altas de anomalias
* Isso pode indicar políticas de bloqueio, falhas técnicas recorrentes ou interferência deliberada

Este tipo de análise seria inviável sem:

* A normalização do ASN
* A modelagem em esquema estrela
* A consolidação dos dados em uma tabela fato analítica

---

## 4. Discussão Geral da Solução

A partir das análises realizadas, é possível afirmar que o MVP:

* Resolve o problema inicial de **opacidade analítica** dos dados do OONI
* Transforma dados JSON semi-estruturados em um **Data Warehouse consultável via SQL**
* Permite análises regionais, temporais e por provedor de forma eficiente

O pipeline construído demonstra, na prática, como técnicas de Engenharia de Dados viabilizam análises que seriam complexas ou inviáveis diretamente sobre dados brutos.

---

## 5. Considerações Finais

A camada de análise atende plenamente aos requisitos do trabalho, cobrindo:

* Qualidade de dados
* Solução das perguntas de negócio
* Discussão analítica conectada ao problema real

Este MVP estabelece uma base sólida para extensões futuras, como:

* Enriquecimento com dados externos de eventos políticos
* Dashboards interativos
* Alertas automatizados de picos de censura
